# Prerequisites

In [69]:
# Install XlsxWriter library
!pip install XlsxWriter

In [70]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns


# Load Data from dolpha.com


## Votes

In [71]:
chain = "polkadot"
url = "https://functions.dolpha.com/governance?chain=" + chain

def load_df(url):
  content_whales = requests.get(url+"&referendum=whalevotes").text
  json_whales = json.loads(content_whales)
  df_whales = pd.DataFrame(json_whales)
  df_whales = df_whales.sort_values(by=['Address', 'Referendum', 'Timestamp'], ascending=[True, True, False])
  df_whales = df_whales.drop_duplicates(subset=['Address', 'Referendum'], keep='first')
  df_whales = df_whales[df_whales['Vote'].isin(['yes', 'no'])]

  content_delegates = requests.get(url+"&referendum=delegates").text
  json_delegates = json.loads(content_delegates)
  df_delegates = pd.DataFrame(json_delegates)
  df_delegates = df_delegates.sort_values(by=['Address', 'Referendum', 'Timestamp'], ascending=[True, True, False])
  df_delegates = df_delegates.drop_duplicates(subset=['Address', 'Referendum'], keep='first')
  df_delegates = df_delegates[df_delegates['Decision'].isin(['yes', 'no'])]
  df_delegates["Vote"] = df_delegates["Decision"]

  # Merge the dataframes on "address", "referendum", and "vote"
  merged_df = pd.merge(df_whales, df_delegates, how="outer", on=['Address', 'Referendum', 'Vote'], suffixes=('_whales', '_delegates'))

  # Sum the "Amount" and "EffectiveVotes" columns respectively
  merged_df['Amount'] = merged_df['Amount_whales'] + merged_df['Amount_delegates']
  merged_df['EffectiveVotes'] = merged_df['EffectiveVotes_whales'].fillna(0) + merged_df['EffectiveVotes_delegates'].fillna(0)


  # Drop the redundant columns
  df = merged_df[['Address', 'Referendum', 'Vote', 'Amount', 'EffectiveVotes']].copy()
  df["Referendum"] = df["Referendum"].astype(int)

  return df

votes_df = load_df(url)

votes_df.to_csv(f"{chain}_votes.csv")
votes_df

,Address,Referendum,Vote,Amount,EffectiveVotes
0,115CGLLXe4SE41MF3vhxvtyhuxTyuVsqPAAV5kz1QcSmVX6,158,no,NaN,105420.00
1,11Q7ismkHUbbUexpQc4DgTvedfsh8jKMDV7jZZoQwv57NLS,132,yes,NaN,7965913.36
2,11gqpAyU17G9EFW5n5MNngh824F3Y2as72V2rgc7Wq5JVRd,175,yes,NaN,15000000.00
3,11gqpAyU17G9EFW5n5MNngh824F3Y2as72V2rgc7Wq5JVRd,222,yes,NaN,5000000.00
4,11gqpAyU17G9EFW5n5MNngh824F3Y2as72V2rgc7Wq5JVRd,81,no,NaN,5000000.00
...,...,...,...,...,...
1945,1v8nuDB4ChEumFThaj7sSySR88nBDmViBJfvhWA2zqmtvY3,81,no,NaN,450000.00
1946,1v8nuDB4ChEumFThaj7sSySR88nBDmViBJfvhWA2zqmtvY3,93,yes,NaN,450000.00
1947,1v8nuDB4ChEumFThaj7sSySR88nBDmViBJfvhWA2zqmtvY3,96,no,NaN,450000.00
1948,1v8nuDB4ChEumFThaj7sSySR88nBDmViBJfvhWA2zqmtvY3,98,no,NaN,450000.00


In [72]:
url

'https://functions.dolpha.com/governance?chain=polkadot'

# Decision Makers

## Fetch Decisions

In [73]:
results_content = requests.get(url+"&referendum=decided").text
results_json = json.loads(results_content)
results_df = pd.json_normalize(results_json)
results_df["yes"] = results_df["decision.yes"]
results_df["no"] = results_df["decision.no"]

results_df = results_df[["index", "yes", "no"]]
results_df.to_csv(f"{chain}_results.csv")

results_df

,index,yes,no
0,239,5.037265e+07,0.000000e+00
1,232,3.039484e+07,5.257517e+04
2,227,3.056551e+07,3.298988e+05
3,221,3.079990e+07,1.634808e+07
4,219,4.434915e+07,2.738190e+06
...,...,...,...
216,4,2.528445e+06,8.001195e+05
217,3,8.585082e+07,0.000000e+00
218,2,3.780598e+05,1.761651e+06
219,1,2.233979e+06,3.249706e+06


## Most significant & Margin voters

We want to understand which voters had the most impact on outcomes. To achieve this, we define two types of voters that voted in the winning direction of a referendum:

- The **significant voters** of a referendum is the smallest possible set of voters that would have been enough to obtain the outcome of a referendum. In other words, they are the smallest group that has at least the same amount of effective votes as the opposing side. They are determined by sorting voters by most effective votes and then taking as many voters as were neccessary to get more effective votes than the losing side had.
- The **margin voters** of a referendum are those that were absolutely neccessary to obtain the outcome of a referendum. They are a subset of significant voters and are determined by sorting voters by most effective votes and then taking as many voters as were neccessary to get more effective votes than the margin of `abs(AYE - NAY)` votes.

It is possible that the smallest voter in each set could also be other voters of the set of all voters that would also satisfy the treshold for the group, but for simplicity we will go along with the algorithms as described.

In [74]:
results_df

,index,yes,no
0,239,5.037265e+07,0.000000e+00
1,232,3.039484e+07,5.257517e+04
2,227,3.056551e+07,3.298988e+05
3,221,3.079990e+07,1.634808e+07
4,219,4.434915e+07,2.738190e+06
...,...,...,...
216,4,2.528445e+06,8.001195e+05
217,3,8.585082e+07,0.000000e+00
218,2,3.780598e+05,1.761651e+06
219,1,2.233979e+06,3.249706e+06


In [75]:
# Correct the threshold calculation: it should be the number of votes the losing side received
results_df['significance_threshold'] = results_df.apply(lambda row: row['no'] if row['yes'] > row['no'] else row['yes'], axis=1)
results_df['margin_treshold'] = (results_df['yes'] - results_df['no']).abs()
results_df['winning_option'] = results_df.apply(lambda row: 'yes' if row['yes'] > row['no'] else 'no', axis=1)
results_df['referendum_id'] = results_df['index']  # Renaming for clarity

# Create a dictionary to map referendum ID to its winning option and threshold
referendum_results = results_df.set_index('referendum_id')[['winning_option', 'significance_threshold', 'margin_treshold']].to_dict('index')

# Initialize an empty DataFrame for the results
significant_votes = pd.DataFrame()

# Iterate over each referendum
for referendum_id, data in referendum_results.items():
    # Filter votes for this referendum and reset index to avoid chaining assignment
    referendum_votes = votes_df[votes_df['Referendum'] == referendum_id].reset_index(drop=True)

    # Sort by vote in the winning direction and by the amount of effective votes
    sorted_votes = referendum_votes[referendum_votes['Vote'] == data['winning_option']].sort_values(by='EffectiveVotes', ascending=False).reset_index(drop=True)

    # Calculate the cumulative sum of effective votes
    sorted_votes['cumulative_votes'] = sorted_votes['EffectiveVotes'].cumsum()
    significance_threshold = data['significance_threshold']
    margin_treshold = data['margin_treshold']

    # Find the index of the first row where the cumulative vote count exceeds the significance_threshold
    if not sorted_votes.empty and sorted_votes['cumulative_votes'].max() > significance_threshold:
        significance_threshold_index = sorted_votes[sorted_votes['cumulative_votes'] > data['significance_threshold']].index[0]

        # Select all rows up to and including the row that crossed the significance_threshold
        significant_votes_of_referendum = sorted_votes.loc[:significance_threshold_index].copy()
    else:
        # If there are no votes or if no votes exceed the significance_threshold, copy the either empty or incomplete data
        significant_votes_of_referendum = sorted_votes.copy()

    # Add the referendum_id to the DataFrame
    significant_votes_of_referendum["margin_voter"] = significant_votes_of_referendum["cumulative_votes"] > margin_treshold
    significant_votes_of_referendum['referendum'] = referendum_id
    significant_votes_of_referendum['significance_threshold'] = significance_threshold
    significant_votes_of_referendum['margin_treshold'] = margin_treshold

    # Use pandas.concat instead of append
    significant_votes = pd.concat([significant_votes, significant_votes_of_referendum], ignore_index=True)


# Filter the DataFrame to only include the necessary columns
significant_votes = significant_votes[['referendum', 'Address', 'EffectiveVotes', "significance_threshold", "margin_treshold", 'cumulative_votes', "margin_voter"]]
significant_votes

,referendum,Address,EffectiveVotes,significance_threshold,margin_treshold,cumulative_votes,margin_voter
0,239,15j4dg5GzsL1bw2U2AWgeyAk6QTxq43V7ZPbXdAmbVLjvDCK,3.211650e+07,0.000000e+00,5.037265e+07,3.211650e+07,False
1,232,16DGiP6jDwAfkAeqGfkUCtheKgUzTy7UeaiFFBAv8BwX3RhN,3.000000e+07,5.257517e+04,3.034227e+07,3.000000e+07,False
2,227,16DGiP6jDwAfkAeqGfkUCtheKgUzTy7UeaiFFBAv8BwX3RhN,3.000000e+07,3.298988e+05,3.023561e+07,3.000000e+07,False
3,221,16DGiP6jDwAfkAeqGfkUCtheKgUzTy7UeaiFFBAv8BwX3RhN,2.340000e+07,1.634808e+07,1.445182e+07,2.340000e+07,True
4,219,16DGiP6jDwAfkAeqGfkUCtheKgUzTy7UeaiFFBAv8BwX3RhN,2.340000e+07,2.738190e+06,4.161096e+07,2.340000e+07,False
...,...,...,...,...,...,...,...
282,3,15j4dg5GzsL1bw2U2AWgeyAk6QTxq43V7ZPbXdAmbVLjvDCK,3.211650e+07,0.000000e+00,8.585082e+07,3.211650e+07,False
283,2,16Zw7drubm8LkaoNSPtNBAaoq2bQAL4m7oySPp4RuXvmETDG,4.442220e+05,3.780598e+05,1.383591e+06,4.442220e+05,False
284,1,15Dy29bn4HmtSfYR6FPqqMUj2wCnn2irouDAep9zsv7iUtNE,1.867756e+06,2.233979e+06,1.015727e+06,1.867756e+06,True
285,1,16Zw7drubm8LkaoNSPtNBAaoq2bQAL4m7oySPp4RuXvmETDG,4.442220e+05,2.233979e+06,1.015727e+06,2.311978e+06,True


## Pivot Tables

In [76]:
# Creating the 4 pivoted dataframes as requested

# 1) significant_voters
significant_voters = significant_votes.groupby('Address').agg(
    margin_voter_count=pd.NamedAgg(column='margin_voter', aggfunc='sum'),
    significant_voter_count=pd.NamedAgg(column='Address', aggfunc='size')
).reset_index()
significant_voters = significant_voters.sort_values(by='margin_voter_count', ascending=False)

# 2) referenda_stats
referenda_stats = significant_votes.groupby('referendum').agg(
    significant_voter_count=pd.NamedAgg(column='Address', aggfunc='nunique'),
    margin_voter_count=pd.NamedAgg(column='margin_voter', aggfunc='sum')
).reset_index()

# 3) count_of_referenda_by_count_of_margin_voters
count_of_referenda_by_count_of_margin_voters = referenda_stats['margin_voter_count'].value_counts().reset_index()
count_of_referenda_by_count_of_margin_voters.columns = ['margin_voter_count', 'referendum_count']

# 4) count_of_referenda_by_count_of_significant_voters
count_of_referenda_by_count_of_significant_voters = referenda_stats['significant_voter_count'].value_counts().reset_index()
count_of_referenda_by_count_of_significant_voters.columns = ['significant_voter_count', 'referendum_count']
count_of_referenda_by_count_of_significant_voters = count_of_referenda_by_count_of_significant_voters.sort_values(by='significant_voter_count', ascending=True)




## Write to Excel

In [77]:
# Create an Excel file where each sheet contains a dataframe
excel_path = f'{chain}_decision_makers.xlsx'
from pandas.io.formats import excel
excel.ExcelFormatter.header_style = None

# Define a function to write a dataframe to an Excel writer with formatted number columns
def write_df(writer, sheet_name, dataframe):
    # Write the dataframe to the workbook
    dataframe.to_excel(writer, sheet_name=sheet_name, index=False)

    # Access the xlsxwriter workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # Define the number format
    number_format = workbook.add_format({'num_format': '#,##0'})

    for i, col in enumerate(dataframe.columns):
        # Find the maximum width of the data in the column
        column_len = max(
            dataframe[col].astype(str).map(len).max(),  # length of data in the column
            len(col)  # length of column header
        )
        # Set the column width
        worksheet.set_column(i, i, column_len, number_format)


with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    # Use the custom function to write each dataframe to a worksheet
    write_df(writer, 'Significant Votes', significant_votes)
    write_df(writer, 'Significant Voters', significant_voters)
    write_df(writer, 'Referenda Stats', referenda_stats)
    write_df(writer, 'Referenda by Margin Voters', count_of_referenda_by_count_of_margin_voters)
    write_df(writer, 'Referenda by Significant Voters', count_of_referenda_by_count_of_significant_voters)
    write_df(writer, "Votes", votes_df)
    write_df(writer, "Results", results_df)
